In [1]:
# from urllib.request import Request, urlopen
from urllib.parse import urlparse
import urllib.robotparser
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as fOptions
from selenium.webdriver.common.by import By
import re
import random
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import psycopg2
import psycopg2.errorcodes as errorcodes
import concurrent.futures
import threading
import logging
# new packages
import mmh3
from ordered_set import OrderedSet
# http = urllib.PoolManager(num_pools=50)

In [2]:
def driver_info(name):
    if name == "Lodi":
#         WEB_DRIVER_LOCATION = "D:/FRI/Summer Semester 2020/Web information extraction and retrieval/Assignment 1/geckodriver-v0.26.0-win64/geckodriver.exe"
#         options = fOptions()
#         driver = webdriver.Firefox(executable_path=WEB_DRIVER_LOCATION,options=options)
        WEB_DRIVER_LOCATION = "D:/FRI/Summer Semester 2020/Web information extraction and retrieval/Assignment 1/geckodriver-v0.26.0-win64/geckodriver.exe"
        options = fOptions()
        # If you comment the following line, a browser will show ...
        options.add_argument("--headless")
        #Adding a specific user agent
        options.add_argument("user-agent=fri-ieps-26")
        driver = webdriver.Firefox(executable_path=WEB_DRIVER_LOCATION,options=options)        
        
        
    elif name == "Viktor":
        WEB_DRIVER_LOCATION = "/home/viktor/Documents/FRI/WIER/Assignments/geckodriver"
        options = fOptions()
        # If you comment the following line, a browser will show ...
        options.add_argument("--headless")
        #Adding a specific user agent
        options.add_argument("user-agent=fri-ieps-26")
        driver = webdriver.Firefox(executable_path=WEB_DRIVER_LOCATION,
                               options=options)
          
    return driver

In [3]:
def pickRandomCoeffs(k):
    randList = []
    
    while k > 0:
        randIndex = random.randint(0, maxShingleID) 

        while randIndex in randList:
            randIndex = random.randint(0, maxShingleID) 

        randList.append(randIndex)
        k = k - 1

    return randList

# For each of the 'numHashes' hash functions, generate a different coefficient 'a' and 'b'.   

In [4]:
TIMEOUT = 5
num_sites = 0
numHashes = 10
maxShingleID = 2**64-1
nextPrime = 1.8446844e+19
coeffA = pickRandomCoeffs(numHashes)
coeffB = pickRandomCoeffs(numHashes)
seed_sites_ids = {}
duplicate_urls = []
lock = threading.Lock()
seed_sites = set(["https://gov.si", "https://evem.gov.si", "https://e-uprava.gov.si", "https://e-prostor.gov.si"])
frontier = OrderedSet() | (seed_sites)
signatures = []
signatureForLink = {}
disallowed_urls = set()
robot_parser = {}
duplicate_binaries = set()
# driver = driver_info("Viktor")
# driver.close()

In [5]:
def connect(user, password, db, host, port):
    conn = psycopg2.connect(host=host, port=port, user=user, password=password, database=db)
    conn.autocommit = True
    return conn

In [6]:
def parse_robots(url):
    print(f"Retrieving robots.txt from '{url}'")
    rp = urllib.robotparser.RobotFileParser()
    rp.set_url(url)
    rp.read()
    robots = None
    try:
        response = requests.get(url, verify=False)
    except Exception as e:
        print(e)
    sitemap_url = None
    if response.status_code == 200:
        robots = response.text.strip()
        reg_sitemap = re.compile("Sitemap:\s(.*)")
        sitemap_url = reg_sitemap.search(robots)
        if(sitemap_url):
            sitemap_url = sitemap_url.group(1)
    return robots, rp, sitemap_url
        

In [7]:
def get_sitemap(url):
    sitemap = None
    logging.info("Get sitemap %s", url)
    if url:
        try:
            response = requests.get(url, verify=False)
            sitemap = response.text.strip()
        except Exception as e:
            print(f"SITEMAP: {e}")
    return sitemap

In [81]:
def get_response_code(url):
    if "mailto:" in url:
        print("skipping: e-mail")
        return 503, None, []
    try:
        response = requests.head(url, verify=False, allow_redirects=True, timeout=1) 
        return response.status_code, response.url, response.history
    except Exception:
        print("Timeout exception for url: ", url)
        return 503, url, []

In [9]:
def check_absolute_url(url):
    return (url.index('://') > 0 or url.index('//') == 0)

In [10]:
data_types_dict = {
    'doc':'DOC',
    'docx':'DOCX',
    'pdf':'PDF',
    'ppt':'PPT',
    'pptx':'PPTX',
}
img_types_dict = {
    'jpg': 'JPG',
    'jpeg': 'JPEG',
    'gif': 'GIF',
    'png': 'PNG'
}

In [11]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

def generate_shingles(tokens):
    shinglesInHTML = set()
    for index in range(0, len(tokens) - 2):
        shingle = "{} {} {}".format(tokens[index], tokens[index + 1], tokens[index + 2])
        hashed_shingle = mmh3.hash64(shingle)[0] & 0xffffffffffffffff
        shinglesInHTML.add(hashed_shingle)
#     print(shinglesInHTML)
    return shinglesInHTML

In [12]:
def minhash(shingleIDSet):

    # The resulting minhash signature for this document. 
    signature = []

    # For each of the random hash functions...
    for i in range(0, numHashes):

        # For each of the shingles actually in the document, calculate its hash code
        # using hash function 'i'. 

        # Track the lowest hash ID seen. Initialize 'minHashCode' to be greater than
        # the maximum possible value output by the hash.
        minHashCode = nextPrime + 1

        # For each shingle in the document...
        for shingleID in shingleIDSet:
            # Evaluate the hash function.
#             print(shingleID.bit_length())
            hashCode = (coeffA[i] * shingleID + coeffB[i]) % nextPrime 

            # Track the lowest hash code seen.
            if hashCode < minHashCode:
                minHashCode = hashCode

        # Add the smallest hash code value as component number 'i' of the signature.
        signature.append(minHashCode)

    # Store the MinHash signature for this document.
    return signature

In [13]:
def compare_signatures(signature):
    t0 = time.time()
    duplicate_signature = None
    count = 0
    # For each of the test documents...
    for signature2 in signatures:
        count = 0
        for k in range(0, numHashes):
            count += (signature[k] == signature2[k])
        if count/numHashes > 0.7:
            break
    print(f"HASH: {count/numHashes}")
    # Calculate the elapsed time (in seconds)
    elapsed = (time.time() - t0)

    print ("Comparing MinHash signatures took %.2fsec" % elapsed)
    return signature2 if count/numHashes > 0.7 else None

In [14]:
site_insert_query = '''INSERT INTO crawldb.site(domain, robots_content, sitemap_content) VALUES (%s,%s,%s) RETURNING id'''
page_insert_query = '''INSERT INTO crawldb.page(site_id, page_type_code, url, html_content, http_status_code, accessed_time) VALUES (%s,%s,%s,%s,%s,%s) RETURNING id'''
page_data_insert_query = ''' INSERT INTO crawldb.page_data(page_id, data_type_code, data) VALUES (%s,%s,%s)'''
image_insert_query = ''' INSERT INTO crawldb.image(page_id, filename, content_type, accessed_time,data) VALUES (%s,%s,%s,%s,%s) RETURNING id'''
link_insert_query = ''' INSERT INTO crawldb.link(from_page, to_page) VALUES (%s,%s)'''

find_page_id_query = '''SELECT id FROM crawldb.page WHERE url = %s'''
update_page_query = ''' UPDATE crawldb.page SET page_type_code=%s, html_content=%s, http_status_code=%s,accessed_time=%s WHERE id=%s'''

In [82]:
def extract_page_info(parent_url):
    
    if re.match("(http|https)://(.*)\.?gov\.si(.*)",parent_url) is None:
        return
    conn = psycopg2.connect(host="localhost", user="postgres", password="postgres")
    conn.autocommit = True
    global frontier;
#     conn =  connect(user="postgres", password="postgres", db="crawldb", host="127.0.0.1", port="5432")
    
    driver = driver_info("Viktor")
    
    accessed_time_parent_url = datetime.now()

    print(f"Retrieving web page URL '{parent_url}'")
    
    driver.get(parent_url)
    
     # Timeout needed for Web page to render (read more about it)
    time.sleep(TIMEOUT)
    
    parsed_url = urlparse(parent_url)
    site_id = seed_sites_ids[parsed_url.netloc]     # DB ID of the current seed site
    html_content_binary = None 
    page_id_parent = None # the value will be set after the parent page is added into DB/updated
    
    # Check if this page is already added to DB
    with lock:
        try:
            cursor = conn.cursor()
            cursor.execute(find_page_id_query,(parent_url,))
            page_id_parent = cursor.fetchone()[0]
            cursor.close()
        except (Exception, psycopg2.DatabaseError) as error :
            print ("Error while trying to fetch page id of: ", parent_url, ": ", error)
    
#     ONLY SEED PAGES
    if page_id_parent is None:
        print("Adding page ", parent_url, " to DB")
       # We need to add the page in the DB
        try:
            http_status_code, http_redirect_url, res_history = get_response_code(parent_url)
            with lock:
                cursor = conn.cursor()
                cursor.execute(page_insert_query, (site_id, "FRONTIER", parent_url, html_content_binary, http_status_code, accessed_time_parent_url))
                page_id_parent = cursor.fetchone()[0]
                cursor.close()
                conn.commit()
        except (Exception, psycopg2.DatabaseError) as error :
            print ("Error while writing page", parent_url, " to DB: ", error)
    
    potential_links = []

    # find all html a tags
    a_tags = driver.find_elements_by_tag_name("a")

    # find all onclick events in every html tag
    onclicks = driver.find_elements(
        By.XPATH,
        "//*[contains(@onclick, 'location.href') or contains(@onclick, 'document.location')]"
    )

    images = driver.find_elements_by_tag_name("img")

    # set of all child links and images
    hrefs = []
    duplicates = []
    image_sources = set()
    binary_urls = set()

    # not needed for now
    parsed_uri = urlparse(parent_url)
    hostname = '{uri.netloc}'.format(uri=parsed_uri)

    # matches the binary files
    binary_file_matcher = re.compile("(\.doc|\.docx|\.pdf|\.pptx|\.ppt|\.png|\.jpg\\.jpeg|\.gif)$")

    for a_tag in a_tags:
        url = a_tag.get_attribute("href")
        with lock:
            potential_links.append(url)
        

    for onclick in onclicks:
        url_elem = onclick.get_attribute("onclick")
        reg_url = re.compile("(document.location|location.href)\s*=\s*[\"|'](.*)[\"|']")
        url = reg_url.search(url_elem).group(1)
        with lock:
            potential_links.append(url)

    for url in potential_links:
        
        accessed_time = datetime.now()
        page_id = None
        add_link_flag = False
        
        if url:
            url = url.replace("http://","https://")
            if url[-1] == "/":
                url = url[:-1]

            gov_si_matcher = re.match("(http|https)://(.*)\.?gov\.si(.*)",url)
            if gov_si_matcher is not None and "mailto:" not in url:
                file_name = url.rsplit("/", 1)[-1]

                # check if the url has hostname, add if not
                if not check_absolute_url(url):
                    if url.startswith("/"):
                        url = "{}{}".format(parent_url, url)
                    else:
                        url = "{}/{}".format(parent_url, url)
                
                # if the url points to binary file add it to the binary files list
                if binary_file_matcher.search(file_name):
                    if url not in duplicate_binaries:
                        binary_urls.add(url)
                        duplicate_binaries.add(url)
                        http_status_code, http_redirect_url, res_history = get_response_code(url)
                        time.sleep(TIMEOUT)
                        # ADD CURRENT BINARY PAGE TO DB
                        try:
                            with lock:
                                cursor = conn.cursor()
                                cursor.execute(page_insert_query, (site_id, "BINARY", url, html_content_binary, http_status_code, accessed_time))
                                page_id = cursor.fetchone()[0]
                                conn.commit()
                                add_link_flag = True
                                cursor.close()

                        except (Exception, psycopg2.DatabaseError) as error :
                            print ("Error while writing binary page", url, " to DB: ", error)

                        # find file extension
                        data_type = file_name.split('.')[-1]

                        # check whether the binary file should be added to images or page_data table
                        if data_type.lower() in data_types_dict and page_id is not None:

                            # ADD CURRENT BINARY PAGE_DATA TO DB
                            data_type_code = data_types_dict[data_type.lower()]

                            try:
                                with lock:
                                    cursor = conn.cursor()
                                    cursor.execute(page_data_insert_query, (page_id, data_type_code, html_content_binary))
                                    conn.commit()
                                    cursor.close()

                            except (Exception, psycopg2.DatabaseError) as error :
                                if error.pgcode == errorcodes.UNIQUE_VIOLATION or error.pgcode == "23505" or "duplicate" in error:
                                        pass
                                else:
                                    print ("Error while writing binary page data", url, " to DB: ", error)

                        if data_type.lower() in img_types_dict and page_id is not None:

                            # ADD CURRENT BINARY IMAGE TO DB
                            content_type = img_types_dict[data_type.lower()]
                            filename= file_name.split('.')[0]

                            try:
                                with lock:
                                    cursor = conn.cursor()
                                    cursor.execute(image_insert_query, (page_id, filename, content_type, accessed_time, html_content_binary))
                                    conn.commit()
                                    cursor.close()

                            except (Exception, psycopg2.DatabaseError) as error :
                                print ("Error while writing image ", url, " to DB: ", error)
                else:
                    if url not in hrefs or url not in frontier:
                        hrefs.append(url)
                        duplicates.append(url)
                        # ADD CURRENT PAGE TO DB
                        try:
                            # This is a non-visited page. The information will be updated once it's visited.
                            content_none = None
                            status_code = None
                            ac_time = None
                            with lock:
                                cursor = conn.cursor()
                                cursor.execute(page_insert_query, (site_id, "FRONTIER", url, content_none, status_code, ac_time))
                                # PROVERI DALI TREBA OBRATNO (COMMIT -> FETCH)
                                page_id = cursor.fetchone()[0]
                                conn.commit()
                                cursor.close()
                                add_link_flag = True
                                
                        except (Exception, psycopg2.DatabaseError) as error :
                            if error.pgcode == "23505":
                                pass
                            else:
                                print ("Error while writing non-visited page ", url, " to DB: ", error)
                    else:
                        if url not in duplicates and url not in duplicate_urls:
                            http_status_code, http_redirect_url, res_history = get_response_code(url)
                            time.sleep(TIMEOUT)
                              # ADD DUPLICATE PAGE TO DB
                            try:
                                with lock:
                                    cursor = conn.cursor()
                                    cursor.execute(page_insert_query, (site_id, "DUPLICATE", url, html_content_binary, http_status_code, accessed_time))
                                    page_id = cursor.fetchone()[0]
                                    conn.commit()
                                    cursor.close()
                                    add_link_flag = True
                                    
                            except (Exception, psycopg2.DatabaseError) as error :
#                                 if error.pgcode == 23505:
                                if error.pgcode == errorcodes.UNIQUE_VIOLATION:
                                    pass
                                else:
                                    print ("Error while writing duplicate page", url, " to DB: ", error.pgcode)
                            with lock:
                                duplicates.append(url)
                    
            if add_link_flag and page_id_parent is not None and page_id is not None:
                # ADD LINKS TO DB
                try:
                    with lock:
                        cursor = conn.cursor()
                        cursor.execute(link_insert_query, (page_id_parent, page_id))
                        conn.commit()
                        cursor.close()

                        
                except (Exception, psycopg2.DatabaseError) as error :
                    print ("Error while writing links", parent_url, " and ", url, " to DB: ", error)
                    
    for image in images:
        src = image.get_attribute("src")
        img_id = None
        page_id = None
        add_link_flag = False
        if src:
            gov_si_matcher = re.match("(http|https)://(.*)\.?gov\.si(.*)",src)
            if gov_si_matcher:
                if src not in duplicate_binaries:
                    image_sources.add(src)
                    duplicate_binaries.add(src)
                    accessed_time = datetime.now()
                    http_status_code, http_redirect_url, res_history = get_response_code(url)
                    time.sleep(TIMEOUT)

                     # ADD CURRENT IMAGE TO PAGE DB
                    try:
                        with lock:
                            cursor = conn.cursor()
                            cursor.execute(page_insert_query, (site_id, "BINARY", src, html_content_binary, http_status_code, accessed_time))
                            page_id = cursor.fetchone()[0]
                            conn.commit()
                            add_link_flag = True
                            cursor.close()

                    except (Exception, psycopg2.DatabaseError) as error :
                        if error.pgcode == errorcodes.UNIQUE_VIOLATION or error.pgcode == "23505" or "duplicate" in error:
                            pass
                        else:
                            print ("Error while writing binary page", src, " to DB: ", error)
                
                if page_id is not None:

                    # ADD CURRENT IMAGE TO IMAGE DB
                    file_name = src.rsplit("/", 1)[-1]
                    filename= file_name.split('.')[0]
                    content_type = file_name.split('.')[-1].lower()
                    if content_type in img_types_dict:
                        content_type = img_types_dict[content_type]
                    else:
                        content_type = None
                    try:
                        with lock:
                            cursor = conn.cursor()
                            cursor.execute(image_insert_query, (page_id, filename, content_type, accessed_time, html_content_binary))
                            img_id = cursor.fetchone()[0]
                            conn.commit()
                            cursor.close()
                            
                    except (Exception, psycopg2.DatabaseError) as error :
                        print ("Error while writing image ", src, " to DB: ", error)

                    if add_link_flag and page_id_parent is not None:
                    # ADD LINKS TO DB
                        try:
                            with lock:
                                cursor = conn.cursor()
                                cursor.execute(link_insert_query, (page_id_parent, page_id))
                                conn.commit()
                                cursor.close()
                        except (Exception, psycopg2.DatabaseError) as error :
                            print ("Error while writing link from : ", parent_url, " to ",src," :", error)

                            
    # FINALLY :)
    # Update the content of the parent_url page 
    
    html = driver.page_source
    
    # extract the text from the raw html (everything below)
    soup = BeautifulSoup(html, "html.parser")
    [
        s.extract()
        for s in soup(['style', 'script', '[document]', 'head', 'title'])
    ]
    text = soup.getText()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    content = '\n'.join(chunk for chunk in chunks if chunk)
    tokens = preprocess(content)
    shingles_in_html = generate_shingles(tokens)
    signature = minhash(shingles_in_html)
    original_signature = compare_signatures(signature)
    code = "HTML"
    
    if original_signature is not None:
        code = "DUPLICATE"
        content = None
        original_signature.sort()
        mapped_sorted_os = map(str, original_signature)
        string_sorted_os = ",".join(mapped_sorted_os)
        original_url = signatureForLink[string_sorted_os]
        try:
            with lock:
                cursor = conn.cursor()
                cursor.execute(find_page_id_query,(original_url,))
                original_parent_id = cursor.fetchone()[0]
                cursor.close()
        except (Exception, psycopg2.DatabaseError) as error :
            print ("Error while trying to fetch page id of: ", original_url, ": ", error)
        try:
            with lock:
                cursor = conn.cursor()
                cursor.execute(link_insert_query, (original_parent_id, page_id_parent))
                conn.commit()
                cursor.close()
        except (Exception, psycopg2.DatabaseError) as error :
            print ("Error while writing duplicate link from : ", original_parent_id, " to ",page_id_parent," :", error)
    else:
        signatures.append(signature)
        signature_copy = signature.copy()
        signature_copy.sort()
        mapped_sorted_os = map(str, signature_copy)
        string_sorted_os = ",".join(mapped_sorted_os)
        signatureForLink[string_sorted_os] = parent_url 
        
    
    if page_id_parent is not None:
        # The page is already in the DB. We need to update the information.
        
        print("UPDATING PARENT: ", parent_url)
        
        http_status_code, http_redirect_url, res_history = get_response_code(parent_url)
        time.sleep(TIMEOUT)
        try:
            with lock:
                cursor = conn.cursor()
                cursor.execute(update_page_query,(code,content,http_status_code,accessed_time_parent_url,page_id_parent))
                cursor.close()
                conn.commit()
                
        except (Exception, psycopg2.DatabaseError) as error :
            print ("ERROR WHILE UPDATING PAGE ", parent_url," : ", error)
    driver.close()
    conn.close()
    return hrefs, image_sources, binary_urls, duplicates, html, content

In [16]:
def start(url):
    if re.match("(http|https)://(.*)\.?gov\.si(.*)", url) is None or "mailto:" in url:
        return
    conn = psycopg2.connect(host="localhost", user="postgres", password="postgres")
    conn.autocommit = True
    url = url.replace("www.", "")
    parsed_url = urlparse(url)
    pt = set(["{uri.scheme}://{uri.netloc}".format(uri=parsed_url),\
              "{uri.scheme}s://{uri.netloc}".format(uri=(parsed_url)),\
              "{uri.scheme}s://www.{uri.netloc}".format(uri=(parsed_url)),\
              "{uri.scheme}://www.{uri.netloc}".format(uri=(parsed_url))])

    intersec = pt & seed_sites
    
    global num_sites
    global frontier
    if len(intersec) == 0 or num_sites < 4:
        try:
            robots_content, rp, sitemap_url = parse_robots("{uri.scheme}://{uri.netloc}/robots.txt".format(uri=(parsed_url)))
            robot_parser[parsed_url.netloc] = rp
            if len(intersec) == 0:
                seed_sites.add(url)
                  # ADD SITE TO DB
            time.sleep(4)
            sitemap_content = get_sitemap(sitemap_url)
            with lock:
                try:
                    cursor = conn.cursor()
                    cursor.execute(site_insert_query, (url, robots_content, sitemap_content))
                    id_of_new_row = cursor.fetchone()[0]
                    conn.commit()
                    cursor.close()
                    # Add the auto-generated ID to the dictionary with seed sites IDs
                    seed_sites_ids[parsed_url.netloc] = id_of_new_row
                except (Exception, psycopg2.DatabaseError) as error :
                    print ("Error while writing site ", url, " to DB: ", error)
        except Exception as e:
            print(f"Seed: {e}")
            return
    else:
        rp = robot_parser[parsed_url.netloc] if robot_parser[parsed_url.netloc] else None
        
    if (rp is not None and rp.can_fetch("fri-ieps-26",url)) or rp is None:
        time.sleep(4)
        try:
            next_links, image_sources, binary_urls, duplicates, http_content, content = extract_page_info(url)
            duplicate_urls.extend(duplicates)
            num_sites += 1
            frontier.update(next_links)
#                 frontier = list(set(frontier))
        except Exception as e:
            print(f"wrong link: {e}")
            return
    else:
        print("rejected", url)
    print(f"Frontier len {len(frontier)}")
    conn.close()
#     print(f"SEEDS:{seed_sites}")

In [90]:
global frontier
# with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:    
#     for i in range(4):
#         executor.submit(start, frontier[i])
        
# minimum = 4
# maximum = 10
# i = 4
while(len(frontier) <= 60000):
    with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:
        print(f"\n ... executing workers ...\n")
        for _ in range(12):
            executor.submit(start, frontier[i])
            i += 1
        print(f"Num sites: {num_sites}, i: {i}")


 ... executing workers ...

Num sites: 7513, i: 7888
Retrieving web page URL 'https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=13741'
Retrieving web page URL 'https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=13632'
Retrieving web page URL 'https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=13715'
Retrieving web page URL 'https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=13720'
Retrieving web page URL 'https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=13777'
Retrieving web page URL 'https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=13733'
Retrieving web page URL 'https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=13649'
Retrieving web page URL 'https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=13642'
Re

/home/viktor/anaconda3/envs/wier/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'e-uprava.gov.si'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/viktor/anaconda3/envs/wier/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'e-uprava.gov.si'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/viktor/anaconda3/envs/wier/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'e-uprava.gov.si'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  I

HASH: 0.0
Comparing MinHash signatures took 0.01sec
UPDATING PARENT:  https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=13777
HASH: 0.7
Comparing MinHash signatures took 0.02sec
UPDATING PARENT:  https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=13741
HASH: 0.9
Comparing MinHash signatures took 0.01sec
UPDATING PARENT:  https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=13689


/home/viktor/anaconda3/envs/wier/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'e-uprava.gov.si'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/viktor/anaconda3/envs/wier/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'e-uprava.gov.si'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


HASH: 0.6
Comparing MinHash signatures took 0.01sec
UPDATING PARENT:  https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=13642
HASH: 0.8
Comparing MinHash signatures took 0.01sec
HASH: 0.1
Comparing MinHash signatures took 0.01sec
UPDATING PARENT:  https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=13826
HASH: 0.8
Comparing MinHash signatures took 0.00sec
UPDATING PARENT:  https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=13720
UPDATING PARENT:  https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=13733


...

HASH: 0.5
Comparing MinHash signatures took 0.01sec
HASH: 0.8
Comparing MinHash signatures took 0.01sec
UPDATING PARENT:  https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=12988
HASH: 0.8
Comparing MinHash signatures took 0.02sec
HASH: 0.1
Comparing MinHash signatures took 0.00sec
UPDATING PARENT:  https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=12979
UPDATING PARENT:  https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=12937
UPDATING PARENT:  https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=4623
HASH: 0.9
Comparing MinHash signatures took 0.00sec
HASH: 0.1

/home/viktor/anaconda3/envs/wier/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'e-uprava.gov.si'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/viktor/anaconda3/envs/wier/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'e-uprava.gov.si'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/viktor/anaconda3/envs/wier/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'e-uprava.gov.si'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  I


Comparing MinHash signatures took 0.01sec
UPDATING PARENT:  https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=4629
UPDATING PARENT:  https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=1105
HASH: 0.6
Comparing MinHash signatures took 0.00sec
UPDATING PARENT:  https://e-uprava.gov.si/drzava-in-druzba/javni-sektor/podrobnosti-institucije.html?id=1836


/home/viktor/anaconda3/envs/wier/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'e-uprava.gov.si'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/viktor/anaconda3/envs/wier/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'e-uprava.gov.si'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/viktor/anaconda3/envs/wier/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'e-uprava.gov.si'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  I

Frontier len 59986
Frontier len 59995
Frontier len 60004
Frontier len 60014
Frontier len 60023
Frontier len 60032
Frontier len 60041
Frontier len 60054
Frontier len 60066
Frontier len 60075
Frontier len 60084
Frontier len 60093


In [ ]:
# start("https://e-uprava.gov.si/")
# start("https://gov.si")
# start("https://evem.gov.si")
# start("https://e-prostor.gov.si")

In [ ]:
# try:
# #     conn =  connect(user="postgres", password="postgres", db="crawldb", host="127.0.0.1", port="5432")
#     conn = psycopg2.connect(host="localhost", user="postgres", password="postgres")
#     conn.autocommit = True
#     cursor = conn.cursor()
#     print("Successfuly connected to crawldb")
# except (Exception, psycopg2.DatabaseError) as error :
#     print ("Error while connecting to DB: ", error)
    
# start("https://e-uprava.gov.si/")
# start("https://gov.si")
# start("https://evem.gov.si")
# start("https://e-prostor.gov.si")
# start("https://e-uprava.gov.si/#mainStarting")

# cursor.close()
# conn.close()
# driver.close()

In [32]:
seed_sites_ids

{'e-uprava.gov.si': 383,
 'evem.gov.si': 384,
 'gov.si': 385,
 'e-prostor.gov.si': 386,
 'entrust.net': 387,
 'fu.gov.si': 404,
 'gis.gov.si': 390,
 'mvn.e-prostor.gov.si': 393,
 'projekt.e-prostor.gov.si': 394,
 'sicas.gov.si': 397,
 'si-trust.gov.si': 400,
 'podatki.gov.si': 402}

In [39]:
signatureForLink_tmp = {}
with open('signatureForLink', "rb") as fp:
    itemlist = pickle.load(fp)
signatureForLink_tmp.update(itemlist)

In [99]:
# Num sites: 1413, i: 1648
import pickle
with open('seed_sites_ids_latest', 'wb') as fp:
    pickle.dump(seed_sites_ids, fp)

In [20]:
test = OrderedSet()
with open ('frontier', 'rb') as fp:
    itemlist = pickle.load(fp)
test.update(itemlist)
print(test[:10])
print(frontier[:10])

OrderedSet(['https://e-prostor.gov.si', 'https://gov.si', 'https://evem.gov.si', 'https://e-uprava.gov.si', 'https://e-uprava.gov.si/#main', 'https://e-uprava.gov.si/#footer', 'https://e-uprava.gov.si/si', 'https://e-uprava.gov.si/it', 'https://e-uprava.gov.si/hu', 'https://e-uprava.gov.si/en'])
OrderedSet(['https://e-prostor.gov.si', 'https://gov.si', 'https://evem.gov.si', 'https://e-uprava.gov.si', 'https://e-uprava.gov.si/#main', 'https://e-uprava.gov.si/#footer', 'https://e-uprava.gov.si/si', 'https://e-uprava.gov.si/it', 'https://e-uprava.gov.si/hu', 'https://e-uprava.gov.si/en'])


In [77]:
# # ZA LISTA
# signatures = []
# with open ('signatures', 'rb') as fp:
#     itemlist = pickle.load(fp)
# signatures.extend(itemlist)

seed_sites = set()
with open ('seed_sites', 'rb') as fp:
    itemlist = pickle.load(fp)
seed_sites.update(itemlist)


In [ ]:
print()